In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('n50.csv')

In [3]:
def number_of_years(y):  # calculates the number of years of the dataset
    p = y.index[0]  # date of first row in the dataset (datetime format)
    q = y.index[len(y) - 1]  # date of last row in the dataset  (datetime format)
    return ((q - p).days + 1) / 365  # the difference give the number of total days (not trading days) over the total number of years in the dataset

In [4]:
def ratio(a, b, c):  # function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
    return (a - c) / b  # a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sharpe)


In [5]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights


In [6]:
mvo_sharpe=pd.DataFrame(columns=['Date', 'Returns', 'Risk','Ratio'])
inputweights=[]

In [7]:
def MVO_Sharpe(start_date,end_date,mvo_sharpe):
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    #trading_days = len(df) / number_of_years(tdf)
    trading_days=1
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio
#     returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sharpe ratio
#     for cols in returnso.columns.tolist():
#         for i in range(0, len(df)):
#             if returnso[cols][i] > 0:
#                 returnso[cols][i] = 0  
    covmatsh = returnsh.cov()
    #covmatso = returnso.cov() * trading_days
    num_portfolios = 50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
    num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358/364
    portfolio_returns = []                    #initializing an empty list for portfolio returns
    portfolio_volatility =[]                  #initializing an empty list for portfolio risk
    stock_weights =[]                         #initializing an empty list for portfolio weights
    sharpe =[]          
    for single_portfolio in range(50000):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
        varsh=np.dot(weights.T,np.dot(covmatsh,weights))
        semi_temp = np.sqrt(varsh)
        shtemp = ratio(returns_temp,semi_temp,risk_free_rate)   #calculating sharpe ratio
        portfolio_returns.append(returns_temp)                       
        portfolio_volatility.append(semi_temp)
        stock_weights.append(weights)
        sharpe.append(shtemp)
    portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Sharpe Ratio' : sharpe}
    for counter,symbol in enumerate(df.columns):
        portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights]
    pc = pd.DataFrame(portfolio)
    pc.loc[:, :] *= 100
    pc['Sharpe Ratio']=pc['Sharpe Ratio']/100 
    max_sharpe=pc['Sharpe Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
    max_sharpe_portfolio=pc.loc[pc['Sharpe Ratio'] == max_sharpe]
    max_weights=max_sharpe_portfolio.values
    max_weights=max_weights[0][3:]
    return max_weights

In [8]:
def MVO_Sharpe_lookback(start_date,end_date,deciding_date,mvo_sharpe,inputweights):
    mvo_sharpe_list=[]
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    #trading_days = len(df) / number_of_years(tdf)
    trading_days=1
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio
    #returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sharpe ratio
#     for cols in returnso.columns.tolist():
#         for i in range(0, len(df)):
#             if returnso[cols][i] > 0:
#                 returnso[cols][i] = 0  
    covmatsh = returnsh.cov() * trading_days  # Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
    #covmatso = returnso.cov() * trading_days  # Annualised covariance matrix calculated wrt returnso i.e. used to calculate sharpe ratio
    num_assets = len(df.columns)  # initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358/364  # initializing risk free rate that will be used in calculating both the ratios (absolute value)
# referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
# In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

# 2021_chen etal_Mean–variance portfolio optimization using machine learning-based stock price prediction
# Repeat the process 50,000times. From a statistical point of view, 50,000 random portfolios cover most possible portfolios with different weights and aresufficiently representative

    global_war_sharpe=[]
    sharpe_portfolio_return=[]
    sharpe_portfolio_risk=[]
    sharpe_portfolio_shratio=[]
    sharpe_portfolio_stockWeights=[]
    fitness=0
    weights=np.array(BALANCE(np.array(inputweights)))
    returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
    varsh=np.dot(weights.T,np.dot(covmatsh,weights))   
    semi_temp = np.sqrt(varsh)      
    fitness = ratio(returns_temp,semi_temp,risk_free_rate)
    sharpe_portfolio_return.append(returns_temp)
    sharpe_portfolio_risk.append(semi_temp)
    sharpe_portfolio_shratio.append(fitness)
    sharpe_portfolio_stockWeights.append(weights)
    sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  
    for counter,symbol in enumerate(df.columns):
      sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
    sharpe_pc = pd.DataFrame(sharpe_portfolio)
    sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
    sharpe_optimal=sharpe_optimal.to_frame()
    sharpe_optimal=sharpe_optimal.transpose()
    sharpe_optimal.loc[:, :] *= 100
    sharpe_optimal.loc[:, 'Sharpe Ratio'] /= 100
    to_append=[deciding_date,sharpe_optimal['Returns'].values[0],sharpe_optimal['Standard Deviation'].values[0],sharpe_optimal['Sharpe Ratio'].values[0]]
    mvo_sharpe_series = pd.Series(to_append, index = mvo_sharpe.columns)
    mvo_sharpe = mvo_sharpe.append(mvo_sharpe_series, ignore_index=True)
    sharpe_optimal.to_csv(str(deciding_date)+"sharpe_optimal.csv")
    print(mvo_sharpe)
    return mvo_sharpe


In [9]:
start_date="2016-01-01"

In [10]:
start_date_index=df.index[df['Date'] == start_date]

In [11]:
end_date="2020-09-30"

In [12]:
end_date_index=df.index[df['Date']== end_date]

In [13]:
start_date_lookback="2020-10-01"

In [14]:
start_date_lookback_index=df.index[df['Date'] == start_date_lookback]

In [15]:
end_date_lookback="2020-12-31"

In [16]:
end_date_lookback_index=df.index[df['Date']== end_date_lookback]


In [17]:
while df.iloc[end_date_index].Date.values[0]:
    inputweights=MVO_Sharpe(df.iloc[start_date_index].Date.values[0],df.iloc[end_date_index].Date.values[0],mvo_sharpe)
    mvo_sharpe=MVO_Sharpe_lookback(df.iloc[start_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index+1].Date.values[0],mvo_sharpe,inputweights)
    start_date_lookback_index+=1
    end_date_lookback_index+=1
    start_date_index+=1
    end_date_index+=1

         Date   Returns      Risk     Ratio
0  2021-01-01  0.355714  0.905805  0.381847
         Date   Returns      Risk     Ratio
0  2021-01-01  0.355714  0.905805  0.381847
1  2021-01-04  0.342598  0.905317  0.367564
         Date   Returns      Risk     Ratio
0  2021-01-01  0.355714  0.905805  0.381847
1  2021-01-04  0.342598  0.905317  0.367564
2  2021-01-05  0.320829  0.861410  0.361029
         Date   Returns      Risk     Ratio
0  2021-01-01  0.355714  0.905805  0.381847
1  2021-01-04  0.342598  0.905317  0.367564
2  2021-01-05  0.320829  0.861410  0.361029
3  2021-01-06  0.294434  0.864757  0.329108
         Date   Returns      Risk     Ratio
0  2021-01-01  0.355714  0.905805  0.381847
1  2021-01-04  0.342598  0.905317  0.367564
2  2021-01-05  0.320829  0.861410  0.361029
3  2021-01-06  0.294434  0.864757  0.329108
4  2021-01-07  0.300227  0.850426  0.341466
         Date   Returns      Risk     Ratio
0  2021-01-01  0.355714  0.905805  0.381847
1  2021-01-04  0.342598  0.90531

IndexError: positional indexers are out-of-bounds

In [19]:
mvo_sharpe.to_csv('mvo_sharpe_dynamic.csv')